In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

# set device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# define hyperparameters
num_epochs = 30
batch_size = 300
learning_rate = 0.0001

# define transform to apply to input data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load CIFAR-10 dataset and apply transform
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

Using device: cpu
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Block
class Block(nn.Module):
    def __init__(self,numInputs, numOutputs,inputChannel, outputchannel):
        super(Block, self).__init__()
        self.Linear = nn.Linear(numInputs, numOutputs)
        self.avgpooling = nn.AvgPool2d(kernel_size=(1, 1))
        self.relu = nn.ReLU()
        self.Convl1 = nn.Conv2d(inputChannel, outputchannel, kernel_size = 3, padding=1,stride=1)

    def forward(self, x):

        # Spatial Avg Pool
        pooling = self.avgpooling(x)

        # Flattning
        flatten_data = pooling.view(pooling.size(0), -1)

        linearOut = self.Linear(flatten_data)

        # RELU
        linearRelU = self.relu(linearOut)

        # ConvLayer * a1
        co = self.Convl1(x)

        mul=0
        for col_idx in range(linearRelU.shape[1]):
          a = linearRelU[:, col_idx]  # pick the column
          mul += a.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)*co
        x = mul
        return x





In [ ]:
class Backbone(nn.Module):
    def __init__(self, num_blocks, numInputs, numOutputs):
        super(Backbone, self).__init__()
        self.num_blocks = num_blocks
        inputChannel = 3
        outputchannel = 5
        for i in range(num_blocks):
          self.add_module('bk{0}'.format(i), Block(numInputs, numOutputs, inputChannel, outputchannel))
          numInputs = outputchannel * 32 *32
          inputChannel = outputchannel
        self.classifier = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Linear(inputChannel, 10))

    def forward(self, x):
      out = x
      for i in range(self.num_blocks):
        out = self._modules['bk{0}'.format(i)](out)
      x = self.classifier(out)
      return x

In [ ]:
class Classifier(nn.Module):
  def __init__(self, in_channels, num_classes):
    super().__init__()
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.fc = nn.Linear(in_channels, num_classes)

  def forward(self, x):
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

In [ ]:
class CIFAR10Model(nn.Module):
    def __init__(self, num_blocks,numInputs, numOutputs):
        super().__init__()
        self.backbone = Backbone(num_blocks,numInputs, numOutputs)

    def forward(self, x):
        x = self.backbone(x)
        return x

In [ ]:
numInputs, numOutputs = 3072, 10

num_blocks=3

model = CIFAR10Model(num_blocks,numInputs,numOutputs).to(device)
# specify loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [ ]:
# train the model
train_losses = []
train_accs = []
val_losses = []
val_accs = []
total_step = len(trainloader)
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Track the loss
        running_loss += loss.item()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}]")

    # Track the training loss and accuracy for this epoch
    train_loss = running_loss / len(trainloader)
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

        # Track the validation loss and accuracy for this epoch
        val_loss = running_loss / len(testloader)
        val_acc = correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")


Epoch [1/30], Step [100/167]
Epoch [1/30], Training Loss: 2.3480, Training Accuracy: 0.0997, Validation Loss: 2.3381, Validation Accuracy: 0.0994
Epoch [2/30], Step [100/167]
Epoch [2/30], Training Loss: 2.3395, Training Accuracy: 0.0999, Validation Loss: 2.3368, Validation Accuracy: 0.0995
Epoch [3/30], Step [100/167]
Epoch [3/30], Training Loss: 2.3379, Training Accuracy: 0.0999, Validation Loss: 2.3361, Validation Accuracy: 0.0997
Epoch [4/30], Step [100/167]
Epoch [4/30], Training Loss: 2.3370, Training Accuracy: 0.0999, Validation Loss: 2.3356, Validation Accuracy: 0.0999
Epoch [5/30], Step [100/167]
Epoch [5/30], Training Loss: 2.3363, Training Accuracy: 0.1000, Validation Loss: 2.3352, Validation Accuracy: 0.0999
Epoch [6/30], Step [100/167]
Epoch [6/30], Training Loss: 2.3357, Training Accuracy: 0.1000, Validation Loss: 2.3347, Validation Accuracy: 0.0999
Epoch [7/30], Step [100/167]
Epoch [7/30], Training Loss: 2.3351, Training Accuracy: 0.1001, Validation Loss: 2.3342, Valida